# Google QUEST Q&A Labeling

The data includes questions and answers from various StackExchange properties. My task is to predict target values of 30 labels for each question-answer pair.

In [20]:
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from nltk.stem import PorterStemmer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [21]:
df=pd.read_csv("D:/data-science/train.csv")
df.head(5)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [22]:
len(df['question_well_written'].unique())

9

In [23]:
len(df['answer_helpful'].unique())

9

In [24]:
len(df['answer_level_of_information'].unique())

9

In [25]:
len(df['answer_plausible'].unique())

9

In [26]:
len(df['answer_relevance'].unique())

9

In [27]:
len(df['answer_satisfaction'].unique())

17

In [28]:
len(df['answer_type_instructions'].unique())

5

In [29]:
len(df['answer_type_procedure'].unique())

5

In [30]:
len(df['answer_type_reason_explanation'].unique())

5

In [31]:
len(df['answer_well_written'].unique())

9

### As column 'question_well_written' has 9 unique values . Similarly all other columns have a specific no of unique values ,So it can be evaluated by classification algorithm.

#### Data Processing
* Remove punctuation
* Tokenization
* Remove stopwards
* Stemming of tokenized words
* Vectorization

In [32]:
df1=pd.read_csv("D:/data-science/train.csv")

In [33]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

#### Out of all independent features ,I've considered only the question_title, question_body, answer to be relevant to what we need.

In [34]:
def remove_puch(text):
    text_nop = "".join([char for char in text if char not in string.punctuation])
    return text_nop
df['question_title'] = df['question_title'].apply(lambda x:remove_puch(x))
df['question_body'] = df['question_body'].apply(lambda x:remove_puch(x))
df['answer'] = df['answer'].apply(lambda x:remove_puch(x))
df.head(1)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,I just got extension tubes so heres the skinny...,rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.0,1.0,0.666667,1.0,1.0,0.8,1.0,0.0,0.0,1.0


In [35]:
import re
def tokenize(text):
    tokens = re.split('\W+',text)
    return tokens
df['question_title'] = df['question_title'].apply(lambda x:tokenize(x.lower()))
df['question_body'] = df['question_body'].apply(lambda x:tokenize(x.lower()))
df['answer'] = df['answer'].apply(lambda x:tokenize(x.lower()))
df.head(1)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,"[what, am, i, losing, when, using, extension, ...","[after, playing, around, with, macro, photogra...",ysap,https://photo.stackexchange.com/users/1024,"[i, just, got, extension, tubes, so, heres, th...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.0,1.0,0.666667,1.0,1.0,0.8,1.0,0.0,0.0,1.0


In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(tokenized_list):
    text = [word for word in tokenized_list if word not in stopword]
    return text
df['answer'] = df['answer'].apply(lambda x:remove_stopwords(x))
df['question_title'] = df['question_title'].apply(lambda x:remove_stopwords(x))
df['question_body'] = df['question_body'].apply(lambda x:remove_stopwords(x))
df.head(1)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,"[losing, using, extension, tubes, instead, mac...","[playing, around, macro, photography, ontheche...",ysap,https://photo.stackexchange.com/users/1024,"[got, extension, tubes, heres, skinny, losing,...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.0,1.0,0.666667,1.0,1.0,0.8,1.0,0.0,0.0,1.0


###### Here I've used porterstemmer to stem the words

In [38]:
ps = nltk.PorterStemmer()
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text
df['answer'] = df['answer'].apply(lambda x:stemming(x))
df['question_title'] = df['question_title'].apply(lambda x:stemming(x))
df['question_body'] = df['question_body'].apply(lambda x:stemming(x))
df.head(1)

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,"[lose, use, extens, tube, instead, macro, len]","[play, around, macro, photographi, onthecheap,...",ysap,https://photo.stackexchange.com/users/1024,"[got, extens, tube, here, skinni, lose, use, t...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.0,1.0,0.666667,1.0,1.0,0.8,1.0,0.0,0.0,1.0


Declaring a function named as "clean_text" which includes all the processes for data cleaning

In [137]:
def clean_text(text):
    text_nop = "".join([char for char in text if char not in string.punctuation])
    tokens = re.split('\W+',text)
    return [word for word in tokens if word not in stopword]

In [138]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer(analyzer = clean_text)

In [139]:
cvec.fit(df1['question_title'])

CountVectorizer(analyzer=<function clean_text at 0x000002BE2D5BA7B8>,
                binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [140]:
ques_tit = cvec.transform(df1['question_title']).todense()
y=df1['answer_level_of_information']
X_train,X_test,y_train,y_test=train_test_split(ques_tit,y,test_size=.2)

In [141]:
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 0)  
  
# fit the regressor with X and Y data 
regressor.fit(X_train, y_train) 

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')

In [142]:
ypred1 = regressor.predict(X_test)
sqrt(mean_squared_error(ypred1,y_test))

0.1385156005895818

In [168]:
y_train1=y_train.astype('int')
y_test1=y_test.astype('int')
accuracy_score(ypred1.astype('int'), y_test1)*100

97.5328947368421

In [143]:
ques_tit1 = pd.DataFrame(ques_tit,columns = cvec.get_feature_names())

In [144]:
cvec.fit(df1['question_body'])
ques_body = pd.DataFrame(cvec.transform(df1['question_body']).todense(),columns = cvec.get_feature_names())

In [145]:
cvec.fit(df1['answer'])
ans = pd.DataFrame(cvec.transform(df1['answer']).todense(),columns = cvec.get_feature_names())


In [146]:
X = pd.concat([ques_tit1, ques_body, ans],axis = 1)

In [147]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2)
from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 0)  
  
# fit the regressor with X and Y data 
regressor.fit(X_train, y_train) 

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')

In [148]:
ypred2 = regressor.predict(X_test)
sqrt(mean_squared_error(ypred1,y_test))

0.13549246569479187

In [170]:
accuracy_score(ypred2.astype('int'), y_test1)*100

97.5328947368421

In [149]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(ques_tit)

In [152]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo')

In [155]:
linclf = svm.LinearSVC()

In [171]:
linclf.fit(X_train, y_train1)

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [172]:
ypred4 = linclf.predict(X_test)

In [173]:
accuracy_score(ypred4.astype('int'), y_test1)*100

97.94407894736842

In [174]:
clf.fit(X_train, y_train1)

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [175]:
ypred5 = linclf.predict(X_test)
accuracy_score(ypred5.astype('int'), y_test1)*100

97.94407894736842

In [164]:
Naive = naive_bayes.MultinomialNB()
y_train1=y_train.astype('int')
y_test1=y_test.astype('int')
Naive.fit(X_train,y_train1)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [182]:
ypred3 = Naive.predict(X_test)
accuracy_score(ypred3, y_test1)*100

97.12171052631578

##### Hence linear support vector machine algorithm supossed to be an great alternave for predicting "answer_level_of_information"

### Similarly we can proceed for rest of the dependent variables